<a href="https://colab.research.google.com/github/hrtywhy/Machine-Learning/blob/main/SGD%2CKNN%2CNaive_Baiyes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Dataset/skripsi/dos_dataset/clean_2.csv')
#df.info()
df=df.drop('Unnamed: 0.1',axis=1)
x=df.iloc[:,df.columns != 'Label']
y=df.iloc[:,-1]
print("x\n",x.info())
y = pd.DataFrame(y)
print('y\n',y.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1245798 entries, 0 to 1245797
Data columns (total 73 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   Unnamed: 0                    1245798 non-null  int64  
 1   Unnamed: 0.1.1                1245798 non-null  int64  
 2    Source IP                    1245798 non-null  int64  
 3    Source Port                  1245798 non-null  int64  
 4    Destination IP               1245798 non-null  int64  
 5    Destination Port             1245798 non-null  int64  
 6    Protocol                     1245798 non-null  int64  
 7    Flow Duration                1245798 non-null  int64  
 8    Total Fwd Packets            1245798 non-null  int64  
 9    Total Backward Packets       1245798 non-null  int64  
 10  Total Length of Fwd Packets   1245798 non-null  float64
 11   Total Length of Bwd Packets  1245798 non-null  float64
 12   Fwd Packet Length Max      

In [6]:
df.describe()

,Unnamed: 0,Unnamed: 0.1.1,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,SYN Flag Count,RST Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Inbound,Label
count,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245753e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06
mean,6.228985e+05,1.074318e+05,2.891303e+09,2.302203e+04,3.166084e+09,3.186662e+04,1.486217e+01,2.202223e+06,3.375339e+00,7.088477e-01,1.069863e+03,3.765930e+02,4.679689e+02,4.562818e+02,4.597351e+02,4.712262e+00,1.475077e+01,1.725317e+00,5.181360e+00,3.952797e+00,inf,inf,2.181317e+05,3.961819e+05,1.065644e+06,9.254274e+02,2.189210e+06,2.936620e+05,4.491561e+05,1.058643e+06,8.675107e+02,9.171877e+05,1.283504e+05,2.349393e+05,5.506912e+05,3.812488e-01,8.118491e-03,-7.708815e+07,-1.122862e+05,1.228033e+06,1.452845e+03,4.562034e+02,4.765040e+02,4.608991e+02,6.727419e+00,2.453121e+03,2.472311e-04,8.118491e-03,1.699826e-01,2.298206e-02,1.019186e-02,8.258803e-02,6.812989e+02,4.597351e+02,5.181360e+00,-7.708815e+07,3.375339e+00,1.069863e+03,7.088477e-01,3.765930e+02,1.205828e+03,1.214158e+02,1.786217e+00,-3.818839e+07,1.085408e+04,9.885855e+03,2.130414e+04,4.560909e+03,8.783779e+05,1.914283e+05,1.043642e+06,7.206579e+05,9.624289e-01,3.339989e+00
std,3.596310e+05,1.089688e+05,1.506559e+08,2.447340e+04,4.198623e+08,1.958453e+04,4.363049e+00,1.051741e+07,1.920314e+02,3.232878e+01,1.441733e+03,5.783615e+04,4.837410e+02,4.815043e+02,4.801285e+02,3.185448e+01,1.730104e+02,1.246812e+01,5.426732e+01,5.142089e+01,NaN,NaN,1.008884e+06,1.777128e+06,5.024904e+06,1.215913e+05,1.050753e+07,1.293623e+06,2.018263e+06,5.021211e+06,1.216486e+05,7.681265e+06,9.316844e+05,1.698214e+06,4.169601e+06,3.506093e+00,8.973621e-02,4.103206e+08,2.468212e+07,9.192624e+05,2.037570e+04,4.814801e+02,5.001346e+02,4.795755e+02,4.907000e+01,3.517655e+04,1.572165e-02,8.973621e-02,3.756177e-01,1.498463e-01,1.004390e-01,3.370081e-01,7.221197e+02,4.801285e+02,5.426732e+01,4.103206e+08,1.920314e+02,1.441733e+03,3.232878e+01,5.783615e+04,3.331159e+03,2.118996e+03,1.814738e+01,1.978021e+08,1.717704e+05,1.473230e+05,2.918235e+05,1.374320e+05,4.404065e+06,1.175897

In [7]:
#normalizating the data
#df.iloc[:,df.columns != ' Flow Packets/s' or df.columns != 'Flow Bytes/s']
normalized_df=(df-df.mean())/df.std()
normalized_df=normalized_df.drop(' Flow Packets/s',axis=1)
normalized_df=normalized_df.drop('Flow Bytes/s',axis=1)

In [8]:
import matplotlib.pyplot as plt
import seaborn as sn
plt.figure(figsize=(100,200))
cor = normalized_df.corr()
sn.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [9]:
#df=df.drop('Unnamed: 0.1',axis=1)
normalized_df.head()

,Unnamed: 0,Unnamed: 0.1.1,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,SYN Flag Count,RST Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Inbound,Label
0,-1.732049,0.186725,-0.030358,1.395596,0.157586,-0.607399,-2.031188,-0.191426,0.003253,0.039938,-0.725421,-0.006304,-0.954992,-0.935156,-0.945029,-0.147931,-0.050579,0.342849,0.015085,-0.076871,-0.178761,-0.175445,-0.174509,-0.007595,-0.190368,-0.178330,-0.168507,-0.173212,-0.007115,-0.119406,-0.137760,-0.138345,-0.132072,0.461697,-0.090471,0.187873,0.004551,-1.335866,-0.070783,-0.935040,-0.940755,-0.948545,-0.137098,-0.069737,-0.015726,-0.090471,2.209740,-0.153371,-0.101473,-0.245062,-0.933777,-0.945029,0.015085,0.187873,0.003253,-0.725421,0.039938,-0.006304,-0.054584,-0.057299,0.066885,0.193064,-0.06319,-0.067103,-0.073003,-0.033187,-0.199447,-0.162793,-0.207939,-0.181785,0.19758,-0.731047
1,-1.732046,-0.373912,-0.030358,0.347887,0.157586,-1.625958,-2.031188,-0.209371,-0.007162,0.039938,-0.733744,-0.006304,-0.954992,-0.935156,-0.945029,-0.147931,-0.050579,0.342849,0.015085,-0.076871,-0.216151,-0.222876,-0.212037,-0.007595,-0.208347,-0.227006,-0.222546,-0.210834,-0.007115,-0.119406,-0.137760,-0.138345,-0.132072,0.461697,-0.090471,0.187873,0.004551,-1.324001,0.465069,-0.935040,-0.940755,-0.948545,-0.137098,-0.069737,-0.015726,-0.090471,2.209740,-0.153371,-0.101473,2.722226,-0.933085,-0.945029,0.015085,0.187873,-0.007162,-0.733744,0.039938,-0.006304,7.027636,-0.057299,-0.043324,0.193064,-0.06319,-0.067103,-0.073003,-0.033187,-0.199447,-0.162793,-0.207939,-0.181785,0.19758,-0.731047
2,-1.732043,0.213485,-0.030358,1.196809,0.157586,-0.560525,-2.031188,-0.193320,0.003253,0.039938,-0.725421,-0.006304,-0.954992,-0.935156,-0.945029,-0.147931,-0.050579,0.342849,0.015085,-0.076871,-0.182709,-0.180450,-0.178468,-0.007595,-0.192263,-0.183461,-0.174204,-0.177179,-0.007115,-0.119406,-0.137760,-0.138345,-0.132072,0.461697,-0.090471,0.187873,0.004551,-1.335864,-0.070722,-0.935040,-0.940755,-0.948545,-0.137098,-0.069737,-0.015726,-0.090471,2.209740,-0.153371,-0.101473,-0.245062,-0.933777,-0.945029,0.015085,0.187873,0.003253,-0.725421,0.039938,-0.006304,-0.054584,-0.057299,0.066885,0.193064,-0.06319,-0.067103,-0.073003,-0.033187,-0.199447,-0.162793,-0.207939,-0.181785,0.19758,-0.731047
3,-1.732040,0.215559,-0.030358,0.892029,0.157586,-1.562081,-2.031188,-0.194976,0.003253,0.039938,-0.725421,-0.006304,-0.954992,-0.935156,-0.945029,-0.147931,-0.050579,0.342849,0.015085,-0.076871,-0.186161,-0.184840,-0.181939,-0.007586,-0.193921,-0.187948,-0.179186,-0.180647,-0.007107,-0.119405,-0.137758,-0.138345,-0.132072,0.746914,-0.090471,0.187873,0.004551,-1.335861,-0.070655,-0.935040,-0.940755,-0.948545,-0.137098,-0.069737,-0.015726,-0.090471,2.209740,-0.153371,-0.101473,-0.245062,-0.933777,-0.945029,0.015085,0.187873,0.003253,-0.725421,0.039938,-0.006304,-0.054584,-0.057299,0.066885,0.193064,-0.06319,-0.067103,-0.073003,-0.033187,-0.199447,-0.162793,-0.207939,-0.181785,0.19758,-0.731047
4,-1.732038,-0.757481,-0.030358

In [10]:
normalized_df=normalized_df.drop('Unnamed: 0', axis=1)
normalized_df=normalized_df.drop('Unnamed: 0.1.1', axis=1) 

In [11]:
y=pd.DataFrame(y)

In [12]:
normalized_df.describe()

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,SYN Flag Count,RST Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Inbound,Label
count,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06
mean,-2.874317e-13,1.054733e-12,1.373855e-13,-4.158871e-15,-1.240177e-11,1.282690e-12,5.754005e-15,1.806123e-13,1.063288e-13,-9.852428e-15,-8.237663e-13,2.961019e-12,-1.186983e-12,-4.358217e-13,8.682086e-13,9.755238e-13,-8.253327e-13,-1.077336e-12,2.138927e-12,2.701441e-12,-2.097049e-12,2.491325e-14,-1.702296e-12,-2.322818e-12,1.352296e-12,1.200137e-12,2.012481e-14,-2.547773e-13,7.891959e-13,-1.325646e-12,2.722098e-12,-5.812431e-13,7.837405e-13,-3.524615e-13,-4.037777e-14,1.950589e-12,-8.555205e-13,3.237758e-12,1.526851e-12,-1.568473e-12,-1.359774e-13,-7.298400e-13,-1.089441e-14,7.837405e-13,-3.928092e-12,-2.100294e-12,-3.022469e-13,-4.219886e-13,1.306471e-12,-1.186983e-12,-8.253385e-13,-3.524615e-13,5.754005e-15,1.063288e-13,1.806123e-13,-9.852428e-15,1.868235e-12,1.265760e-13,-1.197121e-13,-1.845158e-13,-6.030817e-13,1.150594e-12,1.117769e-13,2.976447e-13,1.451334e-12,-1.166242e-12,-1.638095e-12,-2.619898e-12,-2.792804e-12,-4.782956e-12
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.919144e+01,-9.406960e-01,-7.380619e+00,-1.627133e+00,-3.406373e+00,-2.093883

In [13]:
normalized_x=normalized_df.iloc[:,normalized_df.columns != 'Label']

In [14]:
normalized_x.describe()

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,SYN Flag Count,RST Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Inbound
count,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06,1.245798e+06
mean,-2.874317e-13,1.054733e-12,1.373855e-13,-4.158871e-15,-1.240177e-11,1.282690e-12,5.754005e-15,1.806123e-13,1.063288e-13,-9.852428e-15,-8.237663e-13,2.961019e-12,-1.186983e-12,-4.358217e-13,8.682086e-13,9.755238e-13,-8.253327e-13,-1.077336e-12,2.138927e-12,2.701441e-12,-2.097049e-12,2.491325e-14,-1.702296e-12,-2.322818e-12,1.352296e-12,1.200137e-12,2.012481e-14,-2.547773e-13,7.891959e-13,-1.325646e-12,2.722098e-12,-5.812431e-13,7.837405e-13,-3.524615e-13,-4.037777e-14,1.950589e-12,-8.555205e-13,3.237758e-12,1.526851e-12,-1.568473e-12,-1.359774e-13,-7.298400e-13,-1.089441e-14,7.837405e-13,-3.928092e-12,-2.100294e-12,-3.022469e-13,-4.219886e-13,1.306471e-12,-1.186983e-12,-8.253385e-13,-3.524615e-13,5.754005e-15,1.063288e-13,1.806123e-13,-9.852428e-15,1.868235e-12,1.265760e-13,-1.197121e-13,-1.845158e-13,-6.030817e-13,1.150594e-12,1.117769e-13,2.976447e-13,1.451334e-12,-1.166242e-12,-1.638095e-12,-2.619898e-12,-2.792804e-12
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.919144e+01,-9.406960e-01,-7.380619e+00,-1.627133e+00,-3.406373e+00,-2.093883e-01,-1.236954e-02,-2.192621e-02,-7.420674e-01

In [15]:
from sklearn.model_selection import train_test_split
import numpy as np
X_train, X_test, y_train, y_test = train_test_split(normalized_x, y, test_size=0.25, random_state=0)
y_train = np.array(y_train)

In [16]:
normalized_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1245798 entries, 0 to 1245797
Data columns (total 69 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0    Source IP                    1245798 non-null  float64
 1    Source Port                  1245798 non-null  float64
 2    Destination IP               1245798 non-null  float64
 3    Destination Port             1245798 non-null  float64
 4    Protocol                     1245798 non-null  float64
 5    Flow Duration                1245798 non-null  float64
 6    Total Fwd Packets            1245798 non-null  float64
 7    Total Backward Packets       1245798 non-null  float64
 8   Total Length of Fwd Packets   1245798 non-null  float64
 9    Total Length of Bwd Packets  1245798 non-null  float64
 10   Fwd Packet Length Max        1245798 non-null  float64
 11   Fwd Packet Length Min        1245798 non-null  float64
 12   Fwd Packet Length Mean     

In [17]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1245798 entries, 0 to 1245797
Data columns (total 1 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   Label   1245798 non-null  int64
dtypes: int64(1)
memory usage: 9.5 MB


In [18]:
#stochastic gradient descent
from  sklearn.linear_model import SGDClassifier, LinearRegression
from sklearn import metrics
lr = SGDClassifier(loss="log", penalty="l2", max_iter=200)
lr.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier(loss='log', max_iter=200)

In [19]:
y_pred = lr.predict(X_test)
print("Stochastic Gradient Descent\nAccuracy =", metrics.accuracy_score(y_test, y_pred)*100)

Stochastic Gradient Descent
Accuracy = 77.12506020227966


In [20]:
from sklearn import metrics
from sklearn.metrics import f1_score
print('Stochastic Gradient Descent')

print('Accuracy = ', metrics.accuracy_score(y_test, y_pred)*100)
print("Confusion Matrix =\n", metrics.confusion_matrix(y_test, y_pred, labels=None, 
                                              sample_weight=None))
print("Recall =", metrics.recall_score(y_test, y_pred, labels=None, 
                                             pos_label=1, average='weighted', 
                                             sample_weight=None))
print("Classification Report =\n", metrics.classification_report(y_test, y_pred, 
                                                                 labels=None, 
                                                                 target_names=None, 
                                                                 sample_weight=None, 
                                                                 digits=2, 
                                                                 output_dict=False))

print("F1 Score = ",f1_score(y_test, y_pred, average='macro'))

Stochastic Gradient Descent
Accuracy =  77.12506020227966
Confusion Matrix =
 [[14099    15     0     2    54    80    29    42]
 [    0 43034     2     0  6713     2    11     0]
 [    3     9 49202   584    18     5     1     0]
 [    0     1   316 47287   856     0  1814     1]
 [   20 41071     0   119  5557   113    31     7]
 [   40     0     0     4     0 49620     0     0]
 [    2     0     0 18791    14     4 31346    10]
 [   21     0     0   167    63    17   192    61]]
Recall = 0.7712506020227966
Classification Report =
               precision    recall  f1-score   support

           0       0.99      0.98      0.99     14321
           1       0.51      0.86      0.64     49762
           2       0.99      0.99      0.99     49822
           3       0.71      0.94      0.81     50275
           4       0.42      0.12      0.18     46918
           5       1.00      1.00      1.00     49664
           6       0.94      0.62      0.75     50167
           7       0.50    

In [21]:
yt = y_test.iloc[:,0]
yt=np.array(yt)

In [22]:
y_pred[3]

2

In [23]:
y_test = yt
#y_pred = y_pred[:, 0]

In [24]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=8)
model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=8)

In [25]:
y_pred = model.predict(X_test)
print("Accuracy =", metrics.accuracy_score(y_test, y_pred))

Accuracy = 0.8701011398298282


In [26]:
from sklearn import metrics
from sklearn.metrics import f1_score

print('K Nearest Neighbour Classifier')

print('Accuracy = ', metrics.accuracy_score(y_test, y_pred)*100)
print("Confusion Matrix =\n", metrics.confusion_matrix(y_test, y_pred, labels=None, 
                                              sample_weight=None))
print("Recall =", metrics.recall_score(y_test, y_pred, labels=None, 
                                             pos_label=1, average='weighted', 
                                             sample_weight=None))
print("Classification Report =\n", metrics.classification_report(y_test, y_pred, 
                                                                 labels=None, 
                                                                 target_names=None, 
                                                                 sample_weight=None, 
                                                                  digits=2, 
                                                                 output_dict=False))
print("F1 Score = ",f1_score(y_test, y_pred, average='macro'))

K Nearest Neighbour Classifier
Accuracy =  87.01011398298282
Confusion Matrix =
 [[14310     0     0     0     1    10     0     0]
 [    0 36928     2    13 12818     1     0     0]
 [    3     3 49745    61     7     3     0     0]
 [    1     0    69 48868    26     0  1310     1]
 [   11 24409    11    65 22407     1     6     8]
 [    3     0     0     1     1 49657     2     0]
 [    0     0     0  1229    13     1 48886    38]
 [    3     0     0    15    40     3   268   192]]
Recall = 0.8701011398298282
Classification Report =
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     14321
           1       0.60      0.74      0.66     49762
           2       1.00      1.00      1.00     49822
           3       0.97      0.97      0.97     50275
           4       0.63      0.48      0.54     46918
           5       1.00      1.00      1.00     49664
           6       0.97      0.97      0.97     50167
           7       0.80 

In [27]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

In [28]:
pred = model.predict(X_test)

In [29]:
from sklearn import metrics
from sklearn.metrics import f1_score

print('Naive Bayes')

print('Accuracy = ', metrics.accuracy_score(y_test, pred)*100)
print("Confusion Matrix =\n", metrics.confusion_matrix(y_test, y_pred, labels=None, 
                                              sample_weight=None))
print("Recall =", metrics.recall_score(y_test, y_pred, labels=None, 
                                             pos_label=1, average='weighted', 
                                             sample_weight=None))
print("Classification Report =\n", metrics.classification_report(y_test, y_pred, 
                                                                 labels=None, 
                                                                 target_names=None, 
                                                                 sample_weight=None, 
                                                                  digits=2, 
                                                                 output_dict=False))
print("F1 Score = ",f1_score(y_test, y_pred, average='macro'))

Naive Bayes
Accuracy =  75.54503130518543
Confusion Matrix =
 [[14310     0     0     0     1    10     0     0]
 [    0 36928     2    13 12818     1     0     0]
 [    3     3 49745    61     7     3     0     0]
 [    1     0    69 48868    26     0  1310     1]
 [   11 24409    11    65 22407     1     6     8]
 [    3     0     0     1     1 49657     2     0]
 [    0     0     0  1229    13     1 48886    38]
 [    3     0     0    15    40     3   268   192]]
Recall = 0.8701011398298282
Classification Report =
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     14321
           1       0.60      0.74      0.66     49762
           2       1.00      1.00      1.00     49822
           3       0.97      0.97      0.97     50275
           4       0.63      0.48      0.54     46918
           5       1.00      1.00      1.00     49664
           6       0.97      0.97      0.97     50167
           7       0.80      0.37      0.51